In [ ]:
!pip install datasets
!pip install transformers
!pip install librosa
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!git config --global credential.helper store

In [ ]:
import torch
import numpy as np
from datasets import ClassLabel
import pandas as pd
from IPython.display import display, HTML
import random
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from datasets import load_dataset
from datasets import load_metric
from transformers import HubertForCTC
from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Loading Data Set (asr)

In [ ]:
## ASR
dataset = load_dataset("superb", "asr", revision="2.2.2", split={'train':'train[0:10000]','validation':'validation[0:500]','test':'test[0:500]'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/28539 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2703 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2620 [00:00<?, ? examples/s]

Dataset superb downloaded and prepared to /root/.cache/huggingface/datasets/superb/asr/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 500
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 500
    })
})


In [ ]:
my_ds = dataset.remove_columns(["id", 'chapter_id', "speaker_id"])

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(my_ds["train"].remove_columns(["file", "audio"]))

,text
0,WHEN THEY ARRIVED THERE THEY MARCHED STRAIGHT TO ROME AND AFTER SOME FIGHTING
1,COULD HE WITNESS THE DEFINITE IMPRISONMENT OF HIS GRANDFATHER IN A NARROW BOX COULD HE WATCH THE COVERING EARTH FALL NOISILY IN THAT BLEAK PLACE OF SILENCE WITHOUT DISPLAYING FOR ROBINSON
2,SUBCONSCIOUSLY TELEGRAPHING TO HIS MIND THE FACT THAT THE TEXTURE OF THE PAPER WAS HERS HERS AND SHE MUST BE ONE OF THOSE AROUND HIM ONE OF THOSE CROWDING EITHER THE ROW OF SEATS IN FRONT OR BEHIND
3,A LITTLE VICIOUSLY I FEAR DON'T YOU SEE WHAT I MEAN BY THE SOLIDARITY OF THE CRAFT I WAS AGGRIEVED AGAINST HIM AS THOUGH HE HAD CHEATED ME
4,SAID MISTER POYSER WITH A GOOD NATURED BLANKNESS OF IMAGINATION AS TO THE NATURE OF THE ARRANGEMENT IF I'M CALLED UPON TO SPEAK SIR SAID MISSUS POYSER AFTER GLANCING AT HER HUSBAND WITH PITY AT HIS SOFTNESS YOU KNOW BETTER THAN ME
5,GOOD BYE PRINCESS I WISH YOU HAPPINESS AND CONSOLATION AND HOPE TO MEET YOU AGAIN IN HAPPIER CIRCUMSTANCES IF YOU DON'T WANT TO MAKE ME BLUSH PLEASE DON'T THANK ME BUT THE PRINCESS IF SHE DID NOT AGAIN THANK HIM IN WORDS
6,TO LIE DOWN HERE THIS PLACE HAS A BAD NAME A VERY BAD NAME BUT AS THE PROVERB HAS IT KUNSHI AYAYUKI NI CHIKAYORAZU THE SUPERIOR MAN DOES NOT NEEDLESSLY EXPOSE HIMSELF TO PERIL
7,EITHER BY LOOSENING HIS HOLD OF THE VOLUMES OR RAISING HIS HAND ACCIDENTALLY THE VALET'S HAND PRESSED AGAINST THE OTHER'S SLEEVE AND HE RECEIVED A SHOCK FOR THE FOREARM WAS CLEARLY AN ARTIFICIAL ONE IT WAS AGAINST A WOODEN SURFACE BENEATH THE SLEEVE THAT HIS KNUCKLES STRUCK
8,IT REQUIRED SOME STUDY TO FIND A WAY OUT OF THE TRAP WE HAD RUN OURSELVES INTO TOWARDS THE GROUP OF MOUNTAINS LAST DESCRIBED WHICH NOW LAY TO THE EAST OF US
9,HE WOULD HAVE GUARDED WITH A MORE PRECIOUS INTEREST THE DRAPINGS OF HIS FINAL SLEEP THIS NECESSARY TASK ON WHICH BOBBY HAD STUMBLED HAD MADE THE THICKET LESS CONGENIAL THAN THE HOUSE AS HE WALKED BACK HE FORECASTED WITH A KEEN APPREHENSION HIS APPROACHING ORDEAL


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

my_ds = my_ds.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7fe98c9da4d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(my_ds["train"].remove_columns(["file", "audio"]))

,text
0,the conversation between them at northanger had been of the most unfriendly kind henry's indignation on hearing how catherine had been treated on comprehending his father's views and being ordered to acquiesce in them
1,it will do you good instead of harm
2,it was over then and all claim to the name of carlyle was declared to have been forfeited by the lady isabel forever captain levison folded up the letter and placed it securely in an inner pocket is there any news she asked
3,my gallant ned i replied laughing those were artificial pearls ordinary glass beads whose insides were coated with essence of orient wow
4,and as might have been expected one of the boilers immediately exploded one dense fog of steam filled every part of the vessel while shrieks groans and cries were heard on every hand the saloons and cabins soon had the appearance of a hospital
5,when he was sitting or standing still he swayed back and forth incessantly like a rocking toy at the piano he swayed in time to the music and when he was not playing his body kept up this motion like an empty mill grinding on
6,i never heard but one had climbed it and that was lander jordan when the excise was on his heels half a century back but he that tries it stakes all on head and foot and a wounded bird like thee may not dare that flight
7,master said he we come to beg you to tell why so strange an animal as man was made with what meddlest thou said the dervish is it thy business
8,can be produced at a non hypnotic level a large number of hypnotists including the author has come to believe that hypnosis is a semantic problem in which words are the building blocks to success not just any words
9,the story of cupid and psyche first appears in the works of apuleius a writer of the second century of our era it is therefore of much more recent date than most of the legends of the age of fable it is this that keats alludes to in his ode to psyche


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = my_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=my_ds.column_names["train"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 18,
 "'": 12,
 'a': 17,
 'b': 15,
 'c': 14,
 'd': 16,
 'e': 11,
 'f': 13,
 'g': 24,
 'h': 7,
 'i': 22,
 'j': 6,
 'k': 21,
 'l': 27,
 'm': 26,
 'n': 8,
 'o': 19,
 'p': 1,
 'q': 9,
 'r': 3,
 's': 5,
 't': 2,
 'u': 23,
 'v': 20,
 'w': 25,
 'x': 4,
 'y': 10,
 'z': 0}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [ ]:
import json
with open('/content/drive/MyDrive/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "wav2vec2-base-timit-demo-colab"
#tokenizer.push_to_hub(repo_name)

Create Wav2Vec2 Feature Extractor

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [ ]:
dataset_prepared = my_ds.map(prepare_dataset, remove_columns=my_ds.column_names["train"], num_proc=2)

#0:   0%|          | 0/5000 [00:00<?, ?ex/s]

#1:   0%|          | 0/5000 [00:00<?, ?ex/s]

#1:   0%|          | 0/250 [00:00<?, ?ex/s]

#0:   0%|          | 0/250 [00:00<?, ?ex/s]

#0:   0%|          | 0/250 [00:00<?, ?ex/s]

#1:   0%|          | 0/250 [00:00<?, ?ex/s]

Load and Train Model

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model = HubertForCTC.from_pretrained(
    "ntu-spml/distilhubert", 
    ctc_loss_reduction="mean", 
    pad_token_id= processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

Some weights of HubertForCTC were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/hubert/modeling_hubert.py:1121: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=5,
  #fp16=True,
  gradient_checkpointing=True, 
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=1e-3,
  weight_decay=5e-3,
  warmup_steps=250,
  #save_total_limit=2,
)


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_prepared["train"],
    eval_dataset=dataset_prepared["validation"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3125


Step,Training Loss,Validation Loss,Wer
100,3.995800,3.485285,1.000000
200,2.846200,2.811416,1.000000
300,1.533900,1.331299,0.898493
400,1.025000,1.155200,0.801292
500,0.885800,1.009957,0.748795
600,0.828600,0.903906,0.727263
700,0.748300,0.848841,0.674664
800,0.692800,0.812163,0.664206
900,0.675400,0.791003,0.645750
1000,0.648300,0.777614,0.642879


***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to wav2vec2-base-timit-demo-colab/checkpoint-100
Configuration saved in wav2vec2-base-timit-demo-colab/checkpoint-100/config.json
Model weights saved in wav2vec2-base-timit-demo-colab/checkpoint-100/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-demo-colab/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to wav2vec2-base-timit-demo-colab/checkpoint-200
Configuration saved in wav2vec2-base-timit-demo-colab/checkpoint-200/config.json
Model weights saved in wav2vec2-base-timit-demo-colab/checkpoint-200/pytorch_model.bin
Feature extractor saved in wav2vec2-base-timit-demo-colab/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to wav2vec2-base-timit-demo-colab/checkpoint-300
Configuration saved in wav2vec2-base-timit

In [ ]:
train_result

TrainOutput(global_step=3125, training_loss=0.731006190032959, metrics={'train_runtime': 8511.6368, 'train_samples_per_second': 5.874, 'train_steps_per_second': 0.367, 'total_flos': 1.4270251085008374e+18, 'train_loss': 0.731006190032959, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'epoch': 5.0,
 'eval_loss': 0.6494764685630798,
 'eval_runtime': 41.1675,
 'eval_samples_per_second': 12.146,
 'eval_steps_per_second': 1.53,
 'eval_wer': 0.5139956936327283}

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = dataset_prepared["test"].map(map_to_result, remove_columns=dataset_prepared["test"].column_names)


  0%|          | 0/500 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.478


In [ ]:
show_random_elements(results)

,pred_str,text
0,her eys seemed to regard hin with miled pity her hodiness astranged light glowing fantly upon her frale flesh did not whu miliit the siner who o proached her,her eyes seemed to regard him with mild pity her holiness a strange light glowing faintly upon her frail flesh did not humiliate the sinner who approached her
1,he was till suffering from the suddend disterbeance of the quiet reten of his life,he was still suffering from this sudden disturbance of the quiet routine of his life
2,the athers resented posspoedment but ad was justiscruples the chamed me,the others resented postponement but it was just his scruples that charmed me
3,i me,ay me
4,heageaffens,hedge a fence
5,the browund grownd beneth his leght fo the mospart one touch o black ais put for the hollow to dc at linds of d'artk greadi fon the outer curve and wone litle quivering touch of whit draes he inter ege ed the mandablee,the brown ground beneath is left for the most part one touch of black is put for the hollow two delicate lines of dark gray define the outer curve and one little quivering touch of white draws the inner edge of the mandible
6,the top floore blongs ti m alsmaclearon,the top floor belongs to miles mc laren
7,the frayse and the day and the seen harmaneyesed in a cord,the phrase and the day and the scene harmonized in a chord
8,not only this but ond the table i found a smal ball of black dol were clay with spexs ifsomething which loks lick sotust it,not only this but on the table i found a small ball of black dough or clay with specks of something which looks like sawdust in it
9,be myg be a bid aportrye here ind tarbut most of his place was such despred prose,there might be a bit of poetry here and there but most of this place was such desperate prose


In [ ]:
with torch.no_grad():
  logits = model(torch.tensor(dataset_prepared["test"][5:6]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))


/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t h h e | | | m [PAD] [PAD] [PAD] u u [PAD] s e [PAD] [PAD] [PAD] [PAD] i [PAD] t [PAD] | | c [PAD] [PAD] [PAD] [PAD] [PAD] a a [PAD] m e e e | | | n n [PAD] e [PAD] [PAD] a a [PAD] r r r r r r r r r r r r [PAD] | | | [PAD] a n n d [PAD] | h h h e e | | [PAD] r r e [PAD] [PAD] [PAD] [PAD] c c [PAD] [PAD] [PAD] [PAD] [PAD] a l l [PAD] [PAD] l e e d | t h h e | | w w w [PAD] [PAD] o r r r r r r r r r r [PAD] [PAD] [PAD] d d d s s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | t h h e e | | w w [PAD] [PAD] o r r r r r [PAD] [PAD] [PAD] d d s s | | o o f f [PAD] [PAD] | | | s s h h [PAD] [PAD] e l l [PAD] l l [PAD] e [PAD] [PAD] [PAD] s s [PAD] [PAD] [PAD] | | | [PAD] f f r r [PAD] [PAD] [PAD] a [PAD] [PAD] c k [PAD] [PAD] [PAD] m m [PAD] e n n t t | a [PAD] [PAD] [PAD] p [PAD] [PAD] [PA